In [1]:
from selenium import webdriver
import os
import time
from datetime import datetime
from datetime import timedelta
from bs4 import BeautifulSoup as bs
import json
import requests
from urllib.request import urlopen
import sys
import pandas as pd

In [30]:
base_url = 'https://www.thefantasyfanatics.com/dfs/pga/ownership_summary.cfm'

resp = requests.get(base_url)

soup = bs(resp.text, 'lxml')

for p in soup.findAll('p'):
    temp_text = p.text
    if temp_text[:3] == 'PGA':
        print(p.find('a').get('href'))

ownership_summary.cfm?viewTID=946
ownership_summary.cfm?viewTID=944
ownership_summary.cfm?viewTID=939
ownership_summary.cfm?viewTID=938
ownership_summary.cfm?viewTID=935


In [2]:
os.chdir('/home/valesco/Datasets/PGA_Data/pga_ownership/')


for tid in range(949,965):
    try:
        if len(str(tid)) == 3:
            tid_str = str(tid)
        elif len(str(tid)) == 2:
            tid_str = '0' + str(tid)
        else:
            tid_str = '00' + str(tid)

        url = 'https://www.thefantasyfanatics.com/dfs/pga/ownership_summary.cfm?viewTID=' + tid_str
        browser = webdriver.Chrome()

        browser.get(url)
        time.sleep(3)

        select_all = browser.find_element_by_xpath('//*[@id="selectAll"]')

        tournament = browser.find_element_by_xpath('//*[@id="page_content"]/div/div/div/h4[1]')

        tourny_text = tournament.text

        contest_table = browser.find_element_by_xpath('//*[@id="page_content"]/div/div/div/table/thead/tr')

        contest_headers = contest_table.find_elements_by_tag_name('th')

        contest_header_ls = []
        for contest_head in contest_headers:
            contest_header_ls.append(contest_head.text)

        contest_df = pd.DataFrame(columns = contest_header_ls)

        contest_table_rows = browser.find_elements_by_xpath('//*[@id="page_content"]/div/div/div/table/tbody/tr')

        for r, row in enumerate(contest_table_rows):
            cell_values = []
            cells = row.find_elements_by_tag_name('td')
            for cell in cells:
                cell_values.append(cell.text)
            contest_df.loc[r] = cell_values


        header = browser.find_element_by_xpath('//*[@id="varianceTable"]/thead/tr')

        header_cols = header.find_elements_by_tag_name('th')

        header_col_ls = []
        for header_col in header_cols:
            header_col_ls.append(header_col.text)

        ownership_df = pd.DataFrame(columns = header_col_ls)

        table = browser.find_element_by_id('varianceTable')

        time.sleep(2)

        rows = browser.find_elements_by_xpath("//*[@id='varianceTable']/tbody/tr")

        for r, row in enumerate(rows):
            cell_values = []
            cells = row.find_elements_by_tag_name('td')
            for cell in cells:
                cell_values.append(cell.text)

            ownership_df.loc[r] = cell_values

        contest_df.to_csv(tourny_text + '_contests.csv')
        ownership_df.to_csv(tourny_text + '_ownership.csv')
        print(tid_str + ' ' + tourny_text + ' succeeded!!')
        browser.close()
    except:
        print(tid_str + ' failed!!')
        browser.close()

949 2017 Nordea Masters succeeded!!
950 2017 Fedex St Jude Classic succeeded!!
951 failed!!
952 failed!!
953 failed!!
954 2017 Lyoness Open succeeded!!
955 2017 US Open succeeded!!
956 2017 Travelers Championship succeeded!!
957 2017 BMW International Open succeeded!!
958 2017 Quicken Loans National succeeded!!
959 2017 Open de France succeeded!!
960 2017 The Greenbrier Classic succeeded!!
961 2017 Irish Open succeeded!!
962 2017 John Deere Classic succeeded!!
963 2017 Aberdeen Asset Management Scottish Open succeeded!!
964 2017 The Open Championship succeeded!!


In [4]:
contest_df

,Buyin,Contest,Entries,Max,6/6%,Display
0,$300,EURO $4.6K 3-Wood [Single Entry],15,1,6.67%,
1,$27,"EURO $8K Fairway [$8,000 Guaranteed]",341,10,9.38%,
2,$5,"EURO $5 Double Up [$1,500 Gtd] (Multi-Entry)",344,10,12.79%,
3,$3,"EURO $1.5K Hybrid [3 Entry Max, $1.5K Gtd]",587,3,8.52%,
4,$3,"EURO $15K Birdie [$15,000 Guaranteed]","5,860",150,8.43%,
5,$1,"EURO $1K Short Game [$1,000 Guaranteed]","1,175",35,7.57%,


In [3]:
browser = webdriver.Chrome()

url = 'https://www.thefantasyfanatics.com/dfs/pga/ownership_summary.cfm?viewTID=928'

browser.get(url)
time.sleep(3)

contest_table = browser.find_element_by_xpath('//*[@id="page_content"]/div/div/div/table/thead/tr')

contest_headers = contest_table.find_elements_by_tag_name('th')

contest_header_ls = []
for contest_head in contest_headers:
    contest_header_ls.append(contest_head.text)
    
contest_df = pd.DataFrame(columns = contest_header_ls)

contest_table_rows = browser.find_elements_by_xpath('//*[@id="page_content"]/div/div/div/table/tbody/tr')

for r, row in enumerate(contest_table_rows):
    cell_values = []
    cells = row.find_elements_by_tag_name('td')
    for cell in cells:
        cell_values.append(cell.text)
    contest_df.loc[r] = cell_values

In [91]:
contest_df

,Buyin,Contest,Entries,Max,6/6%,Display
0,$300,EURO $4.6K 3-Wood [Single Entry],15,1,6.67%,
1,$27,"EURO $8K Fairway [$8,000 Guaranteed]",341,10,9.38%,
2,$5,"EURO $5 Double Up [$1,500 Gtd] (Multi-Entry)",344,10,12.79%,
3,$3,"EURO $1.5K Hybrid [3 Entry Max, $1.5K Gtd]",587,3,8.52%,
4,$3,"EURO $15K Birdie [$15,000 Guaranteed]","5,860",150,8.43%,
5,$1,"EURO $1K Short Game [$1,000 Guaranteed]","1,175",35,7.57%,
6,$0.00,EURO $1K QUARTER ARCADE [JUST $0.25!],"4,699",50,8.92%,


In [56]:
soup

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="" name="description"/>
<meta content="" name="author"/>
<link href="https://www.thefantasyfanatics.com/assets/ico/favicon.ico" rel="shortcut icon"/>
<title>GOLF Tools - DraftKings Ownership Summary | The Fantasy Fanatics</title>
<!-- Bootstrap core CSS -->
<link href="https://www.thefantasyfanatics.com/assets/css/bootstrap.css" rel="stylesheet"/>
<!-- Custom styles for this template -->
<link href="https://www.thefantasyfanatics.com/assets/css/style.css?c=71" rel="stylesheet"/>
<link href="https://www.thefantasyfanatics.com/assets/css/font-awesome.min.css" rel="stylesheet"/>
<link href="https://www.thefantasyfanatics.com/assets/css/tff_styles.css?c=87" rel="stylesheet"/>
<link href="https://cdn.datatables.net/1.10.10/css/jquery.dataTables.min.css" rel="stylesheet" type="text/css"/>
<li